In [ ]:
# @title Library import
!pip install statkit
!pip install shap
!pip install imblearn
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
import os
import sys
import warnings
from collections import Counter
from sklearn.decomposition import PCA
import shap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.feature_selection import (RFECV, SelectKBest, f_classif,
                                       mutual_info_classif)
from sklearn.linear_model import LassoCV, LogisticRegression, Perceptron
from sklearn.metrics import (accuracy_score, auc, f1_score, precision_score,
                             recall_score, roc_auc_score, roc_curve, balanced_accuracy_score)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample, shuffle
from statkit.decision import NetBenefitDisplay
from sklearn.metrics import precision_recall_curve, auc
from sklearn.calibration import calibration_curve
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif,chi2, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgb

# Required imports
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import (
    LogisticRegression,
    PassiveAggressiveClassifier,
    RidgeClassifier,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_selection import RFE


In [ ]:
# @title Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Classifiers
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# K-Nearest Neighbors (KNN)
knn_param_space = {
    'n_neighbors': [3, 5],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree']
}

# AdaBoost Classifier (ABC)
abc_param_space = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
}

# Logistic Regression (LRC)
lr_param_space = {
    'penalty': ['l2'],
    'C': [0.1, 1],
    'solver': ['lbfgs'],
    'max_iter': [100]
}

# Random Forest Classifier (RFC)
rf_param_space = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'max_features': ['sqrt']
}

# Decision Tree Classifier (DTC)
dt_param_space = {
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt']
}

# Support Vector Classifier (SVC)
svc_param_space = {
    'C': [0.1, 1],
    'kernel': ['linear', 'rbf'],
    'probability': [True]
}

# Gradient Boosting Classifier (GBC)
gbc_param_space = {
    'n_estimators': [50],
    'learning_rate': [0.1],
    'max_depth': [3, 5]
}

# Bagging Classifier (BAG)
bg_param_space = {
    'n_estimators': [50],
    'max_samples': [0.5, 1.0]
}

# Multi-layer Perceptron (MLP)
mlp_param_space = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['relu'],
    'solver': ['adam'],
    'max_iter': [200]
}

# Gaussian Naive Bayes (GNB)
gnb_param_space = {
    'var_smoothing': [1e-9, 1e-7]
}

# Extra Trees Classifier (ETC)
etc_param_space = {
    'n_estimators': [50],
    'max_depth': [5, 10],
    'max_features': ['sqrt']
}
dum_param_space = {

}


# Classifiers Dictionary with Predict-Proba Support
classifiers = {
    'DUM': (DummyClassifier(strategy="uniform"),dum_param_space),
    'GNB': (GaussianNB(), gnb_param_space),
    'MLP': (MLPClassifier(random_state=42), mlp_param_space),
    'DTC': (DecisionTreeClassifier(random_state=42), dt_param_space),
    'RFC': (RandomForestClassifier(random_state=42), rf_param_space),
    'KNN': (KNeighborsClassifier(), knn_param_space),
    'ABC': (AdaBoostClassifier(random_state=42), abc_param_space),
    'LRC': (LogisticRegression(random_state=42), lr_param_space),
    'SVC': (SVC(probability=True, random_state=42), svc_param_space),
    'BAG': (BaggingClassifier(random_state=42), bg_param_space),
    'GBC': (GradientBoostingClassifier(random_state=42), gbc_param_space),
    'ETC': (ExtraTreesClassifier(random_state=42), etc_param_space),
}


In [ ]:
# @title Parameters

datasets = [
            'transcriptomics'
            'radiomics+pre_clinical',
            'pre_clinical',
            'radiomics',
            'genomics',
            'post_clinical',
            'radiomics+post_clinical',
            'genomics+post_clinical',
            'radiomics+genomics+post_clinical',
            'radiomics+genomics'
            ]

labels = pd.read_csv("/content/drive/MyDrive/Projects/Ongoing Projects/Radiogenomics/Featrures/titi_features/new/labels_titi.csv")
num_bootstraps = 100
feature_selections = ['mi','anova','lasso']
path = "path_to_features"
backward_stepwise_selection_algorithms = [LogisticRegression(),Perceptron(),DecisionTreeClassifier(),GradientBoostingClassifier()]

warnings.filterwarnings('ignore')

def append_row(df, row):
    return pd.concat([
                df,
                pd.DataFrame([row], columns=row.index)]
          ).reset_index(drop=True)

def backward_stepwise_selection(X, y, clf):

  # Use RFECV with a RandomForest classifier
  selector = RFE(estimator=clf,n_features_to_select=5)
  selector.fit_transform(X, y)
  features = selector.get_support(indices=True)

  return features

############### Selected Signatures ##################

post_clinical_features = ['Gender', 'Ethnicity', 'Smoking status', '%GG', 'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
                          'Tumor Location (choice=RLL)','Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)']

genomics_features = ['ACTG2', 'AGPS', 'ALG14', 'ANG', 'ANGPTL4', 'ANKRD29', 'ANXA1', 'APOC1', 'APOE']

pre_clinical_features = ['Gender', 'Ethnicity', 'Smoking status', '%GG', 'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
                         'Tumor Location (choice=RLL)','Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)']

radiomics_features = ['original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength',
                      'original_shape_Maximum2DDiameterColumn', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice',
                      'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume', 'original_shape_MinorAxisLength']


In [ ]:
# @title Feature Signature Selection (Corelation + Feature Selection + Backward Stepwise Selection + Classifier)

main_y = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[:66,:]
runs_df = pd.DataFrame()

for dataset in datasets:

  main_X = pd.read_csv(path+dataset+".csv",index_col=0,header=0).iloc[:66,:]

  for feature_selection in feature_selections:
    for backward_stepwise_selection_algorithm in backward_stepwise_selection_algorithms:

      feature_counts = Counter()

      # Shuffle data
      data_shuffled, labels_shuffled = shuffle(main_X, main_y, random_state=42)
      feature_names = data_shuffled.columns

      # Standardize features
      scaler = StandardScaler()
      X_scaled = scaler.fit_transform(data_shuffled)

      # SMOTE resampling
      X_scaled, y = SMOTE(random_state=42).fit_resample(X_scaled,labels_shuffled)

      ########### Corelation + Feature Selection
      for i in range(num_bootstraps):

          # Bootstrap sampling
          X_resampled, y_resampled = resample(X_scaled, y, replace=True, random_state=i)

          # Removing correlated features (|correlation| > 0.90)
          corr_matrix = pd.DataFrame(X_resampled).corr().abs()
          upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
          to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.90)]
          X_resampled = pd.DataFrame(X_resampled).drop(columns=to_drop).values

          # Feature selection
          if feature_selection == 'anova':
            feat_selec = SelectKBest(f_classif, k=10)
            feat_selec.fit_transform(X_resampled, y_resampled)
            selected_features = feat_selec.get_support(indices=True)
            print(main_X.columns[selected_features])

          elif feature_selection == 'mi':
            feat_selec = SelectKBest(mutual_info_classif, k=10)
            feat_selec.fit_transform(X_resampled, y_resampled)
            selected_features = feat_selec.get_support(indices=True)
            print(main_X.columns[selected_features])

          elif feature_selection == 'lasso':
            lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
            lasso.fit(X_resampled, y_resampled)
            selected_features = np.where(lasso.coef_ != 0)[0]
            print(main_X.columns[selected_features])

          print('\r'+str(round(i))+"%", end='', flush=True)

          # Update feature count based on selected features
          feature_counts.update(selected_features)

      print('\r', end='', flush=True)
      most_common_features = [item for item, _ in feature_counts.most_common(10)]
      print("\nSelected features: ",most_common_features)
      selected_features_list = []

      ########### Backward Stepwise Selection
      for i in range(num_bootstraps):

          X_scaled = scaler.fit_transform(X)
          y = main_y

          # SMOTE resampling
          X_scaled, y = SMOTE(random_state=42).fit_resample(X_scaled,y)

          # Bootstrap sampling
          X_resampled, y_resampled = resample(X_scaled[:,most_common_features], y, replace=True, random_state=i)

          X_resampled = pd.DataFrame(X_resampled)
          y_resampled = pd.DataFrame(y_resampled)

          X_resampled = X_resampled.reset_index(drop=True)
          y_resampled = y_resampled.reset_index(drop=True)

          y = pd.Series(y_resampled["Lymphovascular invasion"])

          # Run backward stepwise selection
          selected_features = backward_stepwise_selection(X_resampled, y, backward_stepwise_selection_algorithm)

          print('\r'+str(round(i))+"%", end='', flush=True)

          selected_features_list.append(selected_features)

      print('\r', end='', flush=True)
      # Convert each feature list to a tuple (since lists are not hashable, we use tuples)
      feature_selections_tuples = [tuple(f) for f in selected_features_list]
      # Count the frequency of each feature list
      counter = Counter(feature_selections_tuples)
      # Get the most common feature list and its count
      most_common_feature_list = counter.most_common(1)
      stfeatures = list(most_common_feature_list[0][0])
      print(f"Most selected feature signature: {list(feature_names[stfeatures])}")
      print(f"Repeats: {most_common_feature_list[0][1]}")


      ########### Classifiers
      for name, (clf, param_space) in classifiers.items():

        for i in range(num_bootstraps):

          print('\r'+str(round(i))+"%", end='', flush=True)

          X_train = main_X[list(feature_names[stfeatures])]
          y_train = main_y
          X_train, y_train = shuffle(X_train, y_train, random_state=42)

          X_train_s, y_train_s = resample(X_train, y_train, replace=True, random_state=i)

          scaler = StandardScaler()
          X_train_n = pd.DataFrame(scaler.fit_transform(X_train_s))

          cv_scores = {'ext_accuracy': [], 'accuracy': [],'val_accuracy': [], 'ext_roc_auc': [],'roc_auc': [], 'val_roc_auc': [],
                       'ext_precision': [],'precision': [],'val_precision': [], 'ext_recall': [],'recall': [], 'val_recall': [],'ext_f1': [], 'val_f1': [], 'f1': []}

          kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42,)
          scores = cross_val_score(clf, X_train_n, y_train_s, cv=kf, scoring='roc_auc')
          # Calculate metrics
          cv_scores['roc_auc'].append(np.mean(scores))

        # Average scores across all folds
        print(str(bss_alg.__class__.__name__) +","+ fs + ","+ dataset.split("/")[-1] +","+ name +","+
        str(np.mean(cv_scores['ext_accuracy']))+","+str(np.mean(cv_scores['val_accuracy']))+","+str(np.mean(cv_scores['accuracy']))+","+str(np.mean(cv_scores['ext_roc_auc']))+","+
        str(np.mean(cv_scores['val_roc_auc']))+","+str(np.mean(cv_scores['roc_auc']))+","+str(np.mean(cv_scores['ext_precision']))+","+str(np.mean(cv_scores['val_precision']))+","+
        str(np.mean(cv_scores['precision']))+","+str(np.mean(cv_scores['ext_recall']))+","+str(np.mean(cv_scores['val_recall']))+","+str(np.mean(cv_scores['recall']))+","+","+
        str(np.mean(cv_scores['ext_f1']))+","+str(np.mean(cv_scores['val_f1']))+","+str(np.mean(cv_scores['f1'])))

        run_new_row = pd.Series({"bss_alg" : str(bss_alg.__class__.__name__)
                                 ,"sign" : list(feature_names[stfeatures])
                                 ,"Repeats": most_common_feature_list[0][1]
                                 ,"fs": str(fs)
                                 ,"dataset": dataset.split("/")[-1]
                                 ,"classifier": name
                                 ,"ext_accuracy":str(np.mean(cv_scores['ext_accuracy']))
                                 ,"val_accuracy":str(np.mean(cv_scores['val_accuracy']))
                                 ,"accuracy":str(np.mean(cv_scores['accuracy']))
                                 ,"ext_roc_auc":str(np.mean(cv_scores['ext_roc_auc']))
                                 ,"val_roc_auc":str(np.mean(cv_scores['val_roc_auc']))
                                 ,"roc_auc":str(np.mean(cv_scores['roc_auc']))
                                 ,"ext_precision":str(np.mean(cv_scores['ext_precision']))
                                 ,"val_precision":str(np.mean(cv_scores['val_precision']))
                                 ,"precision":str(np.mean(cv_scores['precision']))
                                 ,"ext_recall":str(np.mean(cv_scores['ext_recall']))
                                 ,"val_recall":str(np.mean(cv_scores['val_recall']))
                                 ,"recall":str(np.mean(cv_scores['recall']))
                                 ,"ext_f1":str(np.mean(cv_scores['ext_f1']))
                                 ,"val_f1":str(np.mean(cv_scores['val_f1']))
                                 ,"f1":str(np.mean(cv_scores['f1']))})

        runs_df = append_row(runs_df, run_new_row)

        runs_df.to_csv("path_to_results.csv")

In [ ]:
# @title First, validating the “best classifier” with the 48 VA hospital patients with NO bootstrapping or any resampling

runs_df = pd.DataFrame()

for dataset in datasets:
  for name, (clf, param_space) in classifiers.items():

    cv_scores = {'ext_accuracy': [], 'accuracy': [],'ext_roc_auc': [],'roc_auc': [],
                  'ext_precision': [],'precision': [], 'ext_recall': [],'recall': [], 'ext_f1': [],  'f1': []}

    X_train = pd.DataFrame()
    X_test = pd.DataFrame()

    y_train = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[:66,:]
    y_test = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[66:,:]

    for ds in dataset.split("+"):
      if ds == "pre_clinical":
        features = pre_clinical_features
      elif ds == "radiomics":
        features = radiomics_features
      elif ds == "genomics":
        features = genomics_features
      elif ds == "post_clinical":
        features = post_clinical_features
      else:
        features = []
      X_train = pd.concat([pd.read_csv(dataset+".csv",index_col=0,header=0)[features],X_train],axis=1).iloc[:66,:]
      X_test = pd.concat([pd.read_csv(dataset+".csv",index_col=0,header=0)[features],X_test],axis=1).iloc[66:,:]

    # Shuffling
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)

    # Normalization
    scaler = StandardScaler()
    X_train_n = pd.DataFrame(scaler.fit_transform(X_train))
    X_test_n = pd.DataFrame(scaler.transform(X_test))

    # fit
    clf.fit(X_train_n, y_train)

    # train predict
    y_train_pred = clf.predict(X_train_n)
    y_train_pred_proba = clf.predict_proba(X_train_n)[:, 1]

    # test predict
    y_pred_test = clf.predict(X_test_n)
    y_pred_test_proba = clf.predict_proba(X_test_n)[:, 1]

    cv_scores['accuracy'].append(accuracy_score(y_train, y_train_pred))
    cv_scores['roc_auc'].append(roc_auc_score(y_train, y_train_pred_proba))
    cv_scores['precision'].append(precision_score(y_train, y_train_pred))
    cv_scores['recall'].append(recall_score(y_train, y_train_pred))
    cv_scores['f1'].append(f1_score(y_train, y_train_pred))

    cv_scores['ext_accuracy'].append(accuracy_score(y_test, y_pred_test))
    cv_scores['ext_roc_auc'].append(roc_auc_score(y_test, y_pred_test_proba))
    cv_scores['ext_precision'].append(precision_score(y_test, y_pred_test))
    cv_scores['ext_recall'].append(recall_score(y_test, y_pred_test))
    cv_scores['ext_f1'].append(f1_score(y_test, y_pred_test))

    # Average scores across all folds
    print(dataset.split("/")[-1] +","+ name +","+
    str(np.mean(cv_scores['ext_accuracy']))+","+str(np.mean(cv_scores['accuracy']))+","+str(np.mean(cv_scores['ext_roc_auc']))+","+
    str(np.mean(cv_scores['roc_auc']))+","+str(np.mean(cv_scores['ext_precision']))+","+
    str(np.mean(cv_scores['precision']))+","+str(np.mean(cv_scores['ext_recall']))+","+str(np.mean(cv_scores['recall']))+","+
    str(np.mean(cv_scores['ext_f1']))+","+str(np.mean(cv_scores['f1'])))


    run_new_row = pd.Series({"dataset": dataset.split("/")[-1]
                            ,"classifier": name
                            ,"ext_accuracy":str(np.mean(cv_scores['ext_accuracy']))
                            ,"accuracy":str(np.mean(cv_scores['accuracy']))
                            ,"ext_roc_auc":str(np.mean(cv_scores['ext_roc_auc']))
                            ,"roc_auc":str(np.mean(cv_scores['roc_auc']))
                            ,"ext_precision":str(np.mean(cv_scores['ext_precision']))
                            ,"precision":str(np.mean(cv_scores['precision']))
                            ,"ext_recall":str(np.mean(cv_scores['ext_recall']))
                            ,"recall":str(np.mean(cv_scores['recall']))
                            ,"ext_f1":str(np.mean(cv_scores['ext_f1']))
                            ,"f1":str(np.mean(cv_scores['f1']))})

    runs_df = append_row(runs_df, run_new_row)

  runs_df.to_csv("model_results.csv")


In [ ]:
# @title Second, I would take a bootstrap size of 48 patients, but without stratifying for LVI+ or LVI-.

res_df = pd.DataFrame()
runs_df = pd.DataFrame()
raws = [False]
featuss = []
for dataset in datasets:
  for name, (clf, param_space) in classifiers2.items():
    for raw in raws:

      cv_scores = {'ext_accuracy': [], 'accuracy': [],'ext_roc_auc': [],'roc_auc': [],
                    'ext_precision': [],'precision': [], 'ext_recall': [],'recall': [], 'ext_f1': [],  'f1': [],  'features': []}

      X_train = pd.DataFrame()
      X_test = pd.DataFrame()

      y_train = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[:66,:]
      y_test = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[66:,:]

      for ds in dataset.split("+"):
        if ds == "pre_clinical":
          features = pre_clinical_features
        elif ds == "radiomics":
          features = radiomics_features
        elif ds == "genomics":
          features = genomics_features
        elif ds == "post_clinical":
          features = post_clinical_features
        else:
          features = []
        if raw:
          X_train = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0),X_train],axis=1).iloc[:66,:]
          X_test = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0),X_test],axis=1).iloc[66:,:]
        else:
          X_train = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0)[features],X_train],axis=1).iloc[:66,:]
          X_test = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0)[features],X_test],axis=1).iloc[66:,:]

      df_X_train = X_train
      df_X_test = X_test

      # Shuffling
      X_train, y_train = shuffle(df_X_train, y_train, random_state=42)
      X_test, y_test = shuffle(df_X_test, y_test, random_state=42)

      X_train, y_train = SMOTE(random_state=42).fit_resample(X_train,y_train)

      # Normalization
      scaler = StandardScaler()
      X_train_n = pd.DataFrame(scaler.fit_transform(X_train))
      X_test_n = pd.DataFrame(scaler.transform(X_test))

      # fit
      grid_search = GridSearchCV(estimator=clf, param_grid=param_space, cv=5, scoring='accuracy')
      grid_search.fit(X_train_n, y_train)
      best_regressor = grid_search.best_estimator_

      shap.initjs()

      explainer = shap.Explainer(best_regressor.predict,X_train_n)


      cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

      accuracy_scores = cross_val_score(best_regressor, X_train_n, y_train, cv=cv, scoring='accuracy')
      roc_auc_scores = cross_val_score(best_regressor, X_train_n, y_train, cv=cv, scoring='roc_auc')
      f1_scores = cross_val_score(best_regressor, X_train_n, y_train, cv=cv, scoring='f1')
      precision_scores = cross_val_score(best_regressor, X_train_n, y_train, cv=cv, scoring='precision')
      recall_scores = cross_val_score(best_regressor, X_train_n, y_train, cv=cv, scoring='recall')

      # train predict
      # y_train_pred = best_regressor.predict(X_train_n)
      # y_train_pred_proba = best_regressor.predict_proba(X_train_n)[:, 1]

      cv_scores['accuracy'].append(accuracy_scores)
      cv_scores['roc_auc'].append(roc_auc_scores)
      cv_scores['precision'].append(precision_scores)
      cv_scores['recall'].append(recall_scores)
      cv_scores['f1'].append(f1_scores)
      cv_scores['features'].append(features)

      for i in range(100):

        print('\r'+str(round(i))+"%", end='', flush=True)

        X_test_s, y_test_s = resample(X_test_n, y_test, replace=True, random_state=100+i)

        # test predict
        y_pred_test = best_regressor.predict(X_test_s)
        y_pred_test_proba = best_regressor.predict_proba(X_test_s)[:, 1]

        shap_values = explainer(X_test_s)
        # print(shap_values.values)
        shap_importance = np.abs(shap_values.values).mean(axis=0)
        shap_importance_df = pd.DataFrame({'Feature': df_X_train.columns, 'Importance': shap_importance})
        top_10_features = shap_importance_df.sort_values(by='Importance', ascending=False).head(10)
        features = top_10_features['Feature'].to_list()
        res_df = pd.concat([res_df,pd.DataFrame(top_10_features)])
        featuss.extend(features)

        res_df.to_csv("feats_rf.csv")

        print(list(y_test_s["Lymphovascular invasion"]))
        print(list(y_pred_test))
        print(list(y_pred_test_proba))

        cv_scores['ext_accuracy'].append(accuracy_score(y_test_s, y_pred_test))
        cv_scores['ext_roc_auc'].append(roc_auc_score(y_test_s, y_pred_test_proba))
        cv_scores['ext_precision'].append(precision_score(y_test_s, y_pred_test))
        cv_scores['ext_recall'].append(recall_score(y_test_s, y_pred_test))
        cv_scores['ext_f1'].append(f1_score(y_test_s, y_pred_test))

      # Average scores across all folds
      print(dataset.split("/")[-1] +","+ name +","+ str(raw)+","+
      str(np.mean(cv_scores['ext_accuracy']))+","+str(np.mean(cv_scores['accuracy']))+","+str(np.mean(cv_scores['ext_roc_auc']))+","+
      str(np.mean(cv_scores['roc_auc']))+","+str(np.mean(cv_scores['ext_precision']))+","+
      str(np.mean(cv_scores['precision']))+","+str(np.mean(cv_scores['ext_recall']))+","+str(np.mean(cv_scores['recall']))+","+
      str(np.mean(cv_scores['ext_f1']))+","+str(np.mean(cv_scores['f1'])))

      run_new_row = pd.Series({"dataset": dataset.split("/")[-1]
                              ,"classifier": name
                              ,"raw": str(raw)
                              ,"ext_accuracy":str(np.mean(cv_scores['ext_accuracy']))
                              ,"ext_accuracy_std":str(np.std(cv_scores['ext_accuracy']))
                              ,"accuracy":str(np.mean(cv_scores['accuracy']))
                              ,"ext_roc_auc":str(np.mean(cv_scores['ext_roc_auc']))
                              ,"ext_roc_auc_std":str(np.std(cv_scores['ext_roc_auc']))
                              ,"roc_auc":str(np.mean(cv_scores['roc_auc']))
                              ,"ext_precision":str(np.mean(cv_scores['ext_precision']))
                              ,"ext_precision_std":str(np.std(cv_scores['ext_precision']))
                              ,"precision":str(np.mean(cv_scores['precision']))
                              ,"ext_recall":str(np.mean(cv_scores['ext_recall']))
                              ,"ext_recall_std":str(np.std(cv_scores['ext_recall']))
                              ,"recall":str(np.mean(cv_scores['recall']))
                              ,"ext_f1":str(np.mean(cv_scores['ext_f1']))
                              ,"ext_f1_std":str(np.std(cv_scores['ext_f1']))
                              ,"f1":str(np.mean(cv_scores['f1']))
                              ,"features":str(features)})

      runs_df = append_row(runs_df, run_new_row)

    runs_df.to_csv("/content/drive/MyDrive/Projects/Ongoing Projects/Radiogenomics/model_results_20.csv")


In [ ]:
# @title Running Classifiers
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
path = "/content/drive/MyDrive/Projects/Ongoing Projects/Radiogenomics/Featrures/titi_features/new/"

runs_df = pd.DataFrame()

# selected signatures
datasets = [
            'pre_clinical',
            'radiomics',
            'genomics',
            'post_clinical',
            'radiomics+pre_clinical',
            # 'radiomics+post_clinical',
            'genomics+post_clinical',
            # 'radiomics+genomics+post_clinical',
            # 'radiomics+genomics'
            ]


post_clinical_features = ['Gender', 'Ethnicity', 'Smoking status', '%GG', 'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
                          'Tumor Location (choice=RLL)','Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)']

genomics_features = ['ACTG2', 'AGPS', 'ALG14', 'ANG', 'ANGPTL4', 'ANKRD29', 'ANXA1', 'APOC1', 'APOE']

pre_clinical_features = ['Gender', 'Ethnicity', 'Smoking status', '%GG', 'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
                         'Tumor Location (choice=RLL)','Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)']

radiomics_features = ['original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength',
                      'original_shape_Maximum2DDiameterColumn', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice',
                      'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume', 'original_shape_MinorAxisLength']

# fig, ax = plt.subplots(figsize=(10, 6))
# trajectories = {
#                  'radiomics+pre_clinical' : ('radiomics+pre_clinical', False, classifiers2["GBC"]),
#                  'pre_clinical' : ('pre_clinical', False, classifiers2["GNB"]),
#                  'radiomics' : ('radiomics', False, classifiers2["DTC"]),
#                  'genomics' : ('genomics', False, classifiers2["SVC"]),
#                  'post_clinical' : ('post_clinical', False, classifiers2["GNB"]),
#                  'radiomics+post_clinical' : ('radiomics+post_clinical', False, classifiers2["GBC"]),
#                  'genomics+post_clinical' : ('genomics+post_clinical', False, classifiers2["GNB"]),
#                  'radiomics+genomics+post_clinical' : ('radiomics+genomics+post_clinical', False, classifiers2["ABC"]),
#                  'radiomics+genomics' : ('radiomics+genomics', False,  classifiers2["SVC"])
#                  }

# trajectories = {
#                  'genomics' : ('genomics', True, classifiers2["GNB"]),
#                  'post_clinical' : ('post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+post_clinical' : ('radiomics+post_clinical', True, classifiers2["LRC"]),
#                  'genomics+post_clinical' : ('genomics+post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+genomics+post_clinical' : ('radiomics+genomics+post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+genomics' : ('radiomics+genomics', True,  classifiers2["GNB"]),
#                 #  'radiomics' : ('radiomics', False, classifiers2["DTC"]),
#                  }

# n_stage = pd.read_csv("post_clinical.csv",index_col=0,header=0).iloc[66:,13].replace(2, 1)

# trajectories = {
#                   # 'radiomics+pre_clinical' : ('radiomics+pre_clinical', True, classifiers2["KNN"]),
#                 #  'pre_clinical' : ('pre_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics' : ('radiomics', False, classifiers2["DTC"]),
#                 #  'genomics' : ('genomics', True, classifiers2["GNB"])
#                 #  'post_clinical' : ('post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+post_clinical' : ('radiomics+post_clinical', True, classifiers2["LRC"]),
#                 #  'genomics+post_clinical' : ('genomics+post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+genomics+post_clinical' : ('radiomics+genomics+post_clinical', True, classifiers2["GNB"]),
#                 #  'radiomics+genomics' : ('radiomics+genomics', True,  classifiers2["GNB"])
#                  }

trajectories = {
                 'radiomics+pre_clinical' : ('radiomics+pre_clinical', True, classifiers2["KNN"]),
                #  'pre_clinical' : ('pre_clinical', True, classifiers2["GNB"]),
                #  'radiomics' : ('radiomics', True, classifiers2["LRC"]),
                #  'radiomics+pre_clinical' : ('radiomics+pre_clinical', False, classifiers2["GBC"]),
                 'pre_clinical' : ('pre_clinical', False, classifiers2["GNB"]),
                 'radiomics' : ('radiomics', False, classifiers2["DTC"]),
                 }


# fig, ax = plt.subplots(figsize=(8, 6))

for trajectory in trajectories.values():

  clf = trajectory[2][0]
  param_space = trajectory[2][1]
  raw = trajectory[1]
  name = trajectory[2][0].__class__.__name__
  dataset = trajectory[0]

  cv_scores = {'ext_accuracy': [], 'accuracy': [],'ext_roc_auc': [],'roc_auc': [],
                'ext_precision': [],'precision': [], 'ext_recall': [],'recall': [], 'ext_f1': [],  'f1': [],  'features': []}

  X_train = pd.DataFrame()
  X_test = pd.DataFrame()

  y_train = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[:66,:]
  y_test = pd.DataFrame(labels["Lymphovascular invasion"]).iloc[66:,:]

  for ds in dataset.split("+"):
    if ds == "pre_clinical":
      features = pre_clinical_features
    elif ds == "radiomics":
      features = radiomics_features
    elif ds == "genomics":
      features = genomics_features
    elif ds == "post_clinical":
      features = post_clinical_features
    else:
      features = []
    if raw:
      X_train = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0),X_train],axis=1).iloc[:66,:]
      X_test = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0),X_test],axis=1).iloc[66:,:]
    else:
      X_train = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0)[features],X_train],axis=1).iloc[:66,:]
      X_test = pd.concat([pd.read_csv(path+dataset+".csv",index_col=0,header=0)[features],X_test],axis=1).iloc[66:,:]

  df_X_train = X_train
  df_X_test = X_test

  # Shuffling
  X_train, y_train = shuffle(df_X_train, y_train, random_state=42)
  X_test, y_test = shuffle(df_X_test, y_test, random_state=42)

  X_train, y_train = SMOTE(random_state=42).fit_resample(X_train,y_train)

  # Normalization
  scaler = StandardScaler()
  X_train_n = pd.DataFrame(scaler.fit_transform(X_train))
  X_test_n = pd.DataFrame(scaler.transform(X_test))

  # fit
  grid_search = GridSearchCV(estimator=clf, param_grid=param_space, cv=5, scoring='roc_auc')
  grid_search.fit(X_train_n, y_train)
  best_regressor = grid_search.best_estimator_

  # '''
  # shap.initjs()

  # explainer = shap.Explainer(best_regressor.predict,X_train_n)
  # shap_values = explainer(X_train_n,max_evals=1000)
  # shap_importance = np.abs(shap_values.values).mean(axis=0)
  # shap_importance_df = pd.DataFrame({'Feature': df_X_train.columns, 'Importance': shap_importance})
  # top_10_features = shap_importance_df.sort_values(by='Importance', ascending=False).head(10)
  # features = top_10_features['Feature'].to_list()

  # new = []
  # for x in features:
  #   x = x.replace("original_shape_","")
  #   # wrapper = textwrap.TextWrapper(width=10)
  #   # word_list = wrapper.wrap(text=x)
  #   # n_w_l = ''
  #   # for word in word_list[:2]:
  #   #   n_w_l = n_w_l + "\n"+word
  #   new.append(x)

  # # plt.figure(figsize=(8, 6))

  # # Plot the bar plot
  # plt.figure(figsize=(20, 10))  # Adjust figure size as needed
  # plt.bar(new, top_10_features['Importance'],color='blue')
  # plt.xlabel('SHAP Importance')
  # plt.ylabel('Feature')
  # plt.title('Top 10 Important Features (SHAP)')
  # # lt.gca().invert_yaxis()  # Invert y-axis for top-to-bottom order
  # plt.show()
  # '''

  # train predict
  y_train_pred = best_regressor.predict(X_train_n)
  y_train_pred_proba = best_regressor.predict_proba(X_train_n)[:, 1]

  cv_scores['accuracy'].append(accuracy_score(y_train, y_train_pred))
  cv_scores['roc_auc'].append(roc_auc_score(y_train, y_train_pred_proba))
  cv_scores['precision'].append(precision_score(y_train, y_train_pred))
  cv_scores['recall'].append(recall_score(y_train, y_train_pred))
  cv_scores['f1'].append(f1_score(y_train, y_train_pred))
  cv_scores['features'].append(features)

  all_y = []
  all_preds = []
  all_preds_proba = []

  for i in range(100):

    print('\r'+str(round(i))+"%", end='', flush=True)

    X_test_s, y_test_s = resample(X_test_n, y_test, replace=False, random_state=100+i,stratify=y_test)

    # test predict
    y_pred_test = best_regressor.predict(X_test_s)
    y_pred_test_proba = best_regressor.predict_proba(X_test_s)[:, 1]

    # print(list(y_test_s["Lymphovascular invasion"]))
    # print(list(y_pred_test))
    # print(list(y_pred_test_proba))



    all_y.extend(list(y_test_s['Lymphovascular invasion']))
    all_preds.extend(list(y_pred_test))
    all_preds_proba.extend(list(y_pred_test_proba))

  print(all_y)
  print(all_preds)
  print(all_preds_proba)

  cv_scores['ext_accuracy'].append(accuracy_score(all_y, all_preds))
  cv_scores['ext_roc_auc'].append(roc_auc_score(all_y, all_preds_proba))
  cv_scores['ext_precision'].append(precision_score(all_y, all_preds))
  cv_scores['ext_recall'].append(recall_score(all_y, all_preds))
  cv_scores['ext_f1'].append(f1_score(all_y, all_preds))

  # Average scores across all folds
  print(dataset.split("/")[-1] +","+ name +","+ str(raw)+","+
  str(np.mean(cv_scores['ext_accuracy']))+","+str(np.mean(cv_scores['accuracy']))+","+str(np.mean(cv_scores['ext_roc_auc']))+","+
  str(np.mean(cv_scores['roc_auc']))+","+str(np.mean(cv_scores['ext_precision']))+","+
  str(np.mean(cv_scores['precision']))+","+str(np.mean(cv_scores['ext_recall']))+","+str(np.mean(cv_scores['recall']))+","+
  str(np.mean(cv_scores['ext_f1']))+","+str(np.mean(cv_scores['f1'])))

  fpr, tpr, thresholds = roc_curve(all_y, all_preds_proba)
  roc_auc = auc(fpr, tpr)
  name = name.replace("KNeighborsClassifier", "KNN").replace("DecisionTreeClassifier","DTC").replace("GaussianNB","GNB")

  plt.plot(fpr, tpr, label=name+" / "+dataset.replace("genomics","TF").replace("radiomics","RF").replace("post_clinical","PostOp_CF").replace("pre_clinical","PreOp_CF")+"\n"+f"(AUC = {roc_auc:.2f}) ",linestyle='-.', lw=2 )
  plt.fill_between(fpr, tpr, alpha=0.1)

  # y_scores_jittered = y_pred_test_proba + np.random.normal(0, 0.2, len(y_pred_test_proba))

  # display = NetBenefitDisplay.from_predictions(np.array(list(y_test_s["Lymphovascular invasion"])), y_pred_test_proba, name=dataset,thresholds=np.linspace(0, 1, 100))
  # ax.grid(True)
  # if dataset=='radiomics+genomics':
  #   display.plot(True,ax=ax)
  # else:
  #   display.plot(False,ax=ax)

  run_new_row = pd.Series({"dataset": dataset.split("/")[-1]
                          ,"classifier": name
                          ,"raw": str(raw)
                          ,"ext_accuracy":str(np.mean(cv_scores['ext_accuracy']))
                          ,"accuracy":str(np.mean(cv_scores['accuracy']))
                          ,"ext_roc_auc":str(np.mean(cv_scores['ext_roc_auc']))
                          ,"roc_auc":str(np.mean(cv_scores['roc_auc']))
                          ,"ext_precision":str(np.mean(cv_scores['ext_precision']))
                          ,"precision":str(np.mean(cv_scores['precision']))
                          ,"ext_recall":str(np.mean(cv_scores['ext_recall']))
                          ,"recall":str(np.mean(cv_scores['recall']))
                          ,"ext_f1":str(np.mean(cv_scores['ext_f1']))
                          ,"f1":str(np.mean(cv_scores['f1']))
                          ,"features":str(features)})

  runs_df = append_row(runs_df, run_new_row)

# runs_df.to_csv("model_results_15.csv")
# Plot the ROC curve
plt.plot([0, 1], [0, 1], color='grey', linestyle='--', lw=1)  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('(ROC) Curve --> PreOperative Prediction')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# @title XAI - SHAP analysis

df = pd.read_csv('gen_all.csv').iloc[:9,:]
# df = pd.read_csv('gen_sel.csv').iloc[:9,:]

# Create the plot with custom figure size
plt.figure(figsize=(6, 10))

# Horizontal bar plot
plt.barh(df['gene'][::-1], df['mean'][::-1]/0.036, xerr=df['std'][::-1]/0.036, capsize=5, color=(251/255, 0/255, 61/255), edgecolor='black',zorder=3)
# plt.barh(df['gene'][::-1], df['mean'][::-1]/0.162, xerr=df['std'][::-1]/0.162, capsize=5, color=(15/255, 115/255, 248/255), edgecolor='black')

# Add labels and title
plt.xlabel('Mean',fontsize=20)
plt.ylabel('TF',fontsize=20)
# plt.title('Horizontal Bar Plot with Grid')

# Add grid behind bars
plt.grid(axis='x', linestyle='--', alpha=0.7, zorder=0)
plt.gca().patch.set_facecolor('white')  # Optional: ensures grid visibility with light bar colors
plt.xlim(0,1.3)

plt.xticks(fontsize=12)
plt.yticks(fontsize=20)

# Adjust bar z-order to be above the grid
for bar in plt.gca().patches:
    bar.set_zorder(2)

# Show the plot
plt.show()
